# TPC3: Analisador Léxico para SPARQL

Este trabalho implementa um tokenizador (analisador léxico) para queries SPARQL, capaz de reconhecer:
- Palavras-chave (SELECT, WHERE, PREFIX, OPTIONAL, FILTER)
- Variáveis (?nome, ?desc)
- Identificadores (:Pessoa, :temIdade)
- Strings, números e operadores
- Pontuação e estrutura da query

Exemplo de Query

# DBPedia: obras de Chuck Berry

select ?nome ?desc where { 
    ?s a dbo:MusicalArtist. 
    ?s foaf:name "Chuck Berry"@en . 
    ?w dbo:artist ?s. 
    ?w foaf:name ?nome. 
    ?w dbo:abstract ?desc 
} LIMIT 1000


In [2]:
import re

def tpc3(teste):
    tokens=[
    ('NEWLINE',r'\n'),
    ('SKIP',r'[ \t]+'),
    ('PREFIX',r'PREFIX\b'),
    ('SELECT',r'SELECT\b'),
    ('WHERE',r'WHERE\b'),
    ('OPTIOMAL',r'OPTIONAL\b'),
    ('FILTER',r'FILTER\b'),
    ('VAR',r'\?[a-zA-Z_][\w]*'),
    ('URI',r'<[^>]*>'),
    ('IDENT',r':[a-zA-Z_][\w]*'),
    ('INT',r'\d+'),
    ('STRING',r'"[^"]*"'),
    ('OP',r'[=!<>]+'),
    ('PUNCT',r'[{}.;,]'),
    ('ERROR',r'.')
]
    reconhecidos=[]
    linha=1 
    tokenreg='|'.join(f'(?P<{name}>{pattern})'for name , pattern in tokens) 
    val=re.finditer(tokenreg,teste)
    for v in val:
        dic=v.groupdict()
        type=None
        valor=v.group()
        for key in dic:
            if dic[key]:
                tipo=key
                break
        if tipo=='NEWLINE':
            linha+=1
            reconhecidos.append((tipo, valor, linha, v.span()))
        elif tipo != 'SKIP':
            reconhecidos.append((tipo, valor, linha, v.span()))
            
    return reconhecidos
    print (reconhecidos)
    
teste="""SELECT ?ola ?jorge ?tudo  WHERE 
{
  ?ola a :Pessoa ;
     :temIdade ?19 ;
     :eIrmaoDe ?jorge .
}
"""
tpc3(teste)

[('SELECT', 'SELECT', 1, (0, 6)),
 ('VAR', '?ola', 1, (7, 11)),
 ('VAR', '?jorge', 1, (12, 18)),
 ('VAR', '?tudo', 1, (19, 24)),
 ('WHERE', 'WHERE', 1, (26, 31)),
 ('NEWLINE', '\n', 2, (32, 33)),
 ('PUNCT', '{', 2, (33, 34)),
 ('NEWLINE', '\n', 3, (34, 35)),
 ('VAR', '?ola', 3, (37, 41)),
 ('ERROR', 'a', 3, (42, 43)),
 ('IDENT', ':Pessoa', 3, (44, 51)),
 ('PUNCT', ';', 3, (52, 53)),
 ('NEWLINE', '\n', 4, (53, 54)),
 ('IDENT', ':temIdade', 4, (59, 68)),
 ('ERROR', '?', 4, (69, 70)),
 ('INT', '19', 4, (70, 72)),
 ('PUNCT', ';', 4, (73, 74)),
 ('NEWLINE', '\n', 5, (74, 75)),
 ('IDENT', ':eIrmaoDe', 5, (80, 89)),
 ('VAR', '?jorge', 5, (90, 96)),
 ('PUNCT', '.', 5, (97, 98)),
 ('NEWLINE', '\n', 6, (98, 99)),
 ('PUNCT', '}', 6, (99, 100)),
 ('NEWLINE', '\n', 7, (100, 101))]

## Implementação do Tokenizador

O tokenizador usa expressões regulares para identificar cada tipo de token na query SPARQL.

## Exemplo de Teste

Vamos testar o tokenizador com uma query SPARQL simples:

In [3]:
# Executar o tokenizador
resultado = tpc3(teste)

# Mostrar os primeiros tokens reconhecidos
for token in resultado[:10]:
    print(token)

('SELECT', 'SELECT', 1, (0, 6))
('VAR', '?ola', 1, (7, 11))
('VAR', '?jorge', 1, (12, 18))
('VAR', '?tudo', 1, (19, 24))
('WHERE', 'WHERE', 1, (26, 31))
('NEWLINE', '\n', 2, (32, 33))
('PUNCT', '{', 2, (33, 34))
('NEWLINE', '\n', 3, (34, 35))
('VAR', '?ola', 3, (37, 41))
('ERROR', 'a', 3, (42, 43))


## Resultado Esperado

Cada token é representado como uma tupla com:
- **Tipo**: categoria do token (SELECT, VAR, PUNCT, etc.)
- **Valor**: texto reconhecido
- **Linha**: número da linha onde o token aparece
- **Posição**: tupla (início, fim) indicando a posição no texto original